In [17]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import mind

In [5]:
# os.chdir('./examples/')
# os.getcwd()

'/Users/joao.guedes/Workspace/metrecs/examples'

In [2]:
SUFFIX = 'small'
PATH_BEHAVIOURS = 'examples/MIND/behaviors_' + SUFFIX + '.tsv'
PATH_ARTICLES = 'examples/MIND/articles_large_narrow.json'

articles_df = mind.read_articles(path=PATH_ARTICLES)
behaviors_presel_df = mind.read_behavior(path=PATH_BEHAVIOURS)

Nr of unique articles 68392


In [3]:
articles_df.tail()

,_c0,newsid,title,category,subcategory,cat_subcat,abstract,publication_date,url,cat_as_list
68387,68273,N15890,Watch Jessica Biel Admit She Was NOT a 'Huge' ...,video,popculture,"[video, popculture]",Jessica was hilariously confronted by Jimmy Fa...,2019-10-23,https://assets.msn.com/labs/mind/AAJd7za.html,[video]
68388,68274,N34020,Anxious Democratic donors reportedly seeking a...,video,news,"[video, news]","""A new report in The New York Times details ho...",2019-10-23,https://assets.msn.com/labs/mind/AAJbrCj.html,[video]
68389,68275,N119744,CNN analyst advises Trump to go back to his fa...,video,news,"[video, news]","""CNN legal analyst and former federal prosecut...",2019-10-23,https://assets.msn.com/labs/mind/AAJbcRA.html,[video]
68390,68276,N119588,Salmon shortage threatening food chain in Paci...,video,news,"[video, news]",CNN's Bill Weir reports.,2019-10-23,https://assets.msn.com/labs/mind/AAJbfNt.html,[video]
68391,68278,N33587,How Myles Garrett and Olivier Vernon are shari...,sports,football_nfl,"[sports, football_nfl]","CLEVELAND, Ohio Prior to this season we wond...",2019-10-22,https://assets.msn.com/labs/mind/AAJ7y4f.html,[sports]


In [4]:
behaviors_presel_df.tail()

,index,user,datetime,behavior_array,preselection_array,pool_array
73147,73148,U77536,11/15/2019 8:40:16 PM,"[N28691, N8845, N58434, N37120, N22185, N60033...","[N496-0, N35159-0, N59856-0, N13270-0, N47213-...","[N496, N35159, N59856, N13270, N47213, N26485,..."
73148,73149,U56193,11/15/2019 1:11:26 PM,"[N4705, N58782, N53531, N46492, N26026, N28088...","[N49285-0, N31958-0, N55237-0, N42844-0, N2986...","[N49285, N31958, N55237, N42844, N29862, N1999..."
73149,73150,U16799,11/15/2019 3:37:06 PM,"[N40826, N42078, N15670, N15295, N64536, N4684...","[N7043-0, N512-0, N60215-1, N45057-0, N496-0, ...","[N7043, N512, N60215, N45057, N496, N37055, N1..."
73150,73151,U8786,11/15/2019 8:29:26 AM,"[N3046, N356, N20483, N46107, N44598, N18693, ...","[N23692-0, N19990-0, N20187-0, N5940-0, N13408...","[N23692, N19990, N20187, N5940, N13408, N31958..."
73151,73152,U68182,11/15/2019 11:54:34 AM,"[N20297, N53568, N4690, N60608, N43709, N43123...","[N29862-0, N5472-0, N21679-1, N6400-0, N53572-...","[N29862, N5472, N21679, N6400, N53572, N50775,..."


In [42]:
users_interactions_count_df = (
    behaviors_presel_df[['user', 'behavior_array']]
    .explode('behavior_array')
    .rename({'behavior_array': 'newsid'}, axis=1)
    .groupby('user')
    .count()
)

users_interactions_count_df.tail()

,newsid
user,
U9990,1
U9994,1
U9996,20
U9998,28
U9999,42


In [46]:
items_interactions_count_df = (
    behaviors_presel_df[['user', 'behavior_array']]
    .explode('behavior_array')
    .rename({'behavior_array': 'newsid'}, axis=1)
    .groupby('newsid')
    .count()
)

items_interactions_count_df = pd.merge(
    articles_df[['newsid']],
    items_interactions_count_df,
    on='newsid',
    how='left'
).fillna(0)

items_interactions_count_df.tail()

,newsid,user
68387,N15890,0.0
68388,N34020,0.0
68389,N119744,0.0
68390,N119588,0.0
68391,N33587,0.0


In [48]:
n_users = behaviors_presel_df['user'].nunique()
n_items = articles_df['newsid'].nunique()
n_interactions = behaviors_presel_df['behavior_array'].explode().nunique()
user_gini = gini_coefficient(users_interactions_count_df['newsid'].values)
item_gini = gini_coefficient(items_interactions_count_df['user'].values)

100%|██████████| 68391/68391 [00:03<00:00, 20767.61it/s]


In [50]:
print ('Number of users:', n_users)
print ('Number of items:', n_items)
print ('Number of interactions:', n_interactions)
print ('Space Log:', space_log(n_users, n_items))
print ('Shape Log:', shape_log(n_users, n_items))
print ('Density Log:', density_log(n_users, n_items, n_interactions))
print ('Overall Sparsity:', 1-10**(density_log(n_users, n_items, n_interactions)))
print ('User Gini:', user_gini)
print ('Item Gini:', item_gini)

Number of users: 50000
Number of items: 68392
Number of interactions: 37682
Space Log: 6.533975308409208
Shape Log: -0.1360352997371709
Density Log: -4.957841363176031
Overall Sparsity: 0.9999889805825243
User Gini: 0.6765973475342618
Item Gini: 0.9596907126510239
